# Use Web API

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.14957314.svg)](https://doi.org/10.5281/zenodo.14957314)

**This notebook is for the Public Sector Data Sciences and Management class at John Glenn College of Public Affairs.**

**Author: Tian Lou**

The Ohio State University

**Citation**: *Lou, T. (2025, March 2). Public Sector Data Science Class: How to Use Web API. Zenodo. https://doi.org/10.5281/zenodo.14957314*


## Table of Contents
<a id = "toc"> </a>

- [1. API Liraries](#lib)
    
    **New Libraries**: `bls`, `census`, `us`, `requests`, and `plotly`
    
    Use `pip install bls census us requests plotly` to install these libraries if you do not have them installed on your computer.
    
    
- [2. BLS API Example](#bls_api)
    - [2.1. Get Adams County's Monthly Unemployment Rates Between 2020 and 2023](#adams_ur) \
    **Key Functions**:`bls.get_series()`, `rename()`, `to_csv()`
    
    - [2.2. Get 88 Ohio Counties' Monthly Unemployment Rates Between 2020 and 2023](#all_cnty_ur) \
    **Key Functions**:`np.arange()`, `str()`, `zfill()`, list comprehension, `for` loop, `pd.concat()`
    
    
- [3. Census API Example](#census_api)
    - [3.1. State Level Foreign-born Population Percentage Change Between 2010 and 2020](#fb_pop) \
    **Key Functions**: `api_key.acs5st.state`, `merge()`
    - [3.2. Ohio School District Level Population Under 18 Years](#schl_dist_pop18) \
    **Key Functions**: `requests`
    
    
- [4. Interactive Maps](#viz)
    - [4.1. Ohio County Unemployment Rates Over Time](#oh_cnty_line) \
    **Key Functions**: `pd.read_csv(, dtype = {})`, `.isin()`, `set_ylim()`, `set_ylabel()`, `set_xlabel()`
    - [4.2. *Percentage Change in Foreign-born Population](#fb_map) \
    **Key Functions**: `px.choropleth()`

## 1. API Libraries
<a id = "lib"> </a>

[Go back to Table of Content](#toc)

Many public data wesites, such as [Bureau of Labor Statistics (BLS)](https://www.bls.gov/developers/home.htm), [Federal Reserve Economic Data (FRED)](https://fred.stlouisfed.org/docs/api/fred/), and [U.S. Census](https://www.census.gov/data/developers/data-sets.html), offer application programming interfaces (APIs) that enable users to conveniently retrieve large amounts of data. It is also easier and more efficient to manage the data in Python after pulling it using the API, compared to manual downloads from the websites. Additionally, many developers have already created Python libraries that facilitate easy API usage with predefined functions. The API libraries we will use in this notebook are [**bls**](https://github.com/OliverSherouse/bls) and [**census**](https://pypi.org/project/census/). At the end of this notebook, we will also create two visualizations by using [**Matplotlib**](https://matplotlib.org/) and [**Plotly**](https://plotly.com/python/).

In [1]:
#pip install bls census us requests plotly

In [2]:
#Import libraries
import pandas as pd
import numpy as np

#BLS API
import bls as bls

#Census API wrapper
from census import Census
from us import states

#For accessing API
import requests

#Data visualization library
import matplotlib.pyplot as plt
import plotly.express as px

## 2. BLS API Example
<a id = "bls_api"> </a>

[Go back to Table of Content](#toc)

In this section, we will use BLS API to retrieve 2020 - 2023 Ohio county level unemployment rates. Before using BLS API, you need to have two things ready.

1) **Create a public data API account and register for a key on [BLS website](https://data.bls.gov/registrationEngine/)**.

2) **Know the series id(s) of the data you want to retrieve.** 

In our example, since we need county level unemployment rates, we can go to the BLS [Local Area Unemployment Statistics (LAUS)](https://www.bls.gov/lau/) wesite first. Then 1) click on "LAUS Data"; 2) select "Database"; and 3) choose "One Screen". A new window will pop up. On this new window, select "39 Ohio", "F Counties and equivalents", and "FCN3900100000000 Adams County, OH" (Not Seasonally Adjusted). Finally, click "Add to selection" and "Get Data". After you go through this process, you should finally see the results showing on the screenshot below. To use the BLS API, we need the **Series IDs**. For example, **LAUCN390010000000003** is the series id of the unemployment rate of Adams County in Ohio. Each measure and each geographical level should have a unique series id on BLS website.

> You may have noticed that the series ids and the column names showing in the table are not in the same order. For example, LAUCN390010000000003 is the unemployment rate, but not the labor force (the first column in the data table). How can we know which series id is for which measure? The easiest way is to just look up the series id on Google.

<div class="alert alert-danger"><strong></strong> Here is an example of the series ids for Ohio Adams County's labor force, employment, unemployment, and unemployment rate. <br></div> 


<div>
<img src = "BLS4.png" width="600"/>
</div>

### 2.1 Get Adams County's Monthly Unemployment Rates Between 2020 and 2023
<a id = "adams_ur"> </a>

[Go back to Table of Content](#toc)

To pull the BLS data via API, we can use the `bls.get_series()` function. We need to define four parameters inside of this function: 

1) the data series id,  "**LAUCN390010000000003**"; 

2) the starting year, "**2020**"; 

3) the ending year, "**2023**"; 

4) your BLS API key.

Outside of the `bls.get_series()` function, we use `pd.DataFrame()` to convert the resulting data into the DataFrame format and use the [`reset_index()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) function to reset the index for the DataFrame so that it is easier for us to use the other functions that we are familiar with. The final DataFrame is saved to `adams_ur_df`.

> Many websites limit the number of times you can use their API each day. This is to avoid abusing the API function. Therefore, before you pull massive amount of data with API, always test with a small number of data points first (such as only pull one year of data for one county) and ensure that your functions and setups are correct. After you pull the data, save them to a csv file and avoid pulling the data over and over again.

><font color='red'> **Before you run the code below, please change YOUR KEY to your own BLS key.** </font>

In [ ]:
#Get Adams County's monthly unemployment rates from 2020 to 2023
adams_ur_df = pd.DataFrame(bls.get_series('LAUCN390010000000003',
                                          2020,
                                          2023, 
                                          key = 'YOUR KEY')).reset_index()

#Check the first five rows of the data
adams_ur_df.head()

The column name `LAUCN390010000000003` is difficult for us to remember and to use for analysis. Here, we can use the `.rename()` function to change column names. Inside the function, we first specify that we want to change column names by using `columns = {}`. Inside of the curly braces, we define a pair of values, which is called [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) in Python. The pair of values have a one-to-one relationship. The code `.rename(columns = {'LAUCN390010000000003' : 'unemployment_rate'})` tells Python to change the column name `LAUCN390010000000003` to `unemployment_rate`. You can check [`.rename()` documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) for more examples.

In [ ]:
#Rename column
adams_ur_df = adams_ur_df.rename(columns = {'LAUCN390010000000003' : 'unemployment_rate'})

#Check the data
adams_ur_df.head()

After pulling the data, we can save it to a csv file (or other data file format you need) by using `.to_csv()` function. Inside the function, you should at least specify the file name. In the code below, I will show you two ways to save the data. In the first method, you can just define the file name, and the file will be saved in the folder where you put your notebook. I do not recommend this method, because over time, you folder will be flooded with messy data files. In the second method, you should first define the directory of the folder in which you want to save the data (usually a folder designated for data). Then use the `+` sign to concatenate the directory string with the file name string inside of the `to_csv()` function. We can also define `index = False`, which is to avoid export the index of a DataFrame. Otherwise, in your final csv file, the first column would be a list of numbers that represent the index. The reasons for saving the data to a data file are 1) some websites limit how many times you can use API each day; and 2) pulling the data via API could be time-consuming, especially if you need to pull a large dataset.

In [ ]:
#Save your data to the same folder (where you put your notebook)
#Not recommend this method because over time, your folder will become messy.
adams_ur_df.to_csv('admas_ur.csv', index = False)

<font color='red'> **Before you run the code below, please change YOUR OWN DIRECTORY to your own directory (the place where you want to put the data on your laptop).**

In [ ]:
#Save your data to a designated folder for this class

#Define the file directory
data_directory = 'YOUR OWN DIRECTORY'

#Save the data
adams_ur_df.to_csv(data_directory + 'admas_ur.csv', index = False)

### Class Activity
Work in groups. Go to the BLS [LAUS website](https://www.bls.gov/lau/). Then get the series id of the labor force for Columbus (Ohio Metropolitan area level). Use the API package to download Columbus labor force data between 2019 and 2021.

### 2.2. Get 88 Ohio Counties' Monthly Unemployment Rates
<a id = "all_cnty_ur"> </a>

[Go back to Table of Content](#toc)

Most of the time, we need to pull more than one county's data. However, you do not need to repeat the above code many times. Instead, you can use a loop to easily get all 88 Ohio counties' data. First, let's take a closer look at the series id and see if we can find any patterns. In the series id "LAUCN390010000000003", **LAUCN** represents the data series. **39** is the [state FIPS code](https://www.bls.gov/respondents/mwr/electronic-data-interchange/appendix-d-usps-state-abbreviations-and-fips-codes.htm) and **001** is the [county FIPS code](https://en.wikipedia.org/wiki/List_of_counties_in_Ohio). The series ids for the 88 counties are mostly the same, except for the three-digit county FIPS code (LAUCN39<font color='red'>**001**</font>0000000003).

Check the [complete list of Ohio county FIPS code](https://en.wikipedia.org/wiki/List_of_counties_in_Ohio). You can see that they are all odd numbers and are between 1 and 175. We can use Python to generate these numbers and add leading zeros to numbers that are less than 100.


<h5><center>Selected Ohio Counties' FIPS Codes</center></h5>

| County Name | FIPS Code |
| :-------- | ------- |
| Adams | 001 |
| Allen | 003 |
| Cuyahoga | 035 |
| Franklin | 049 |
| Montgomery | 113 |
| Wyandot | 175 |

The code below creates a list of county FIPS codes. It is more complicated than the code we have used in the previous notebook. I suggest you to try each part of the code in a new code cell and explore what they mean. The code `[str(x).zfill(3) for x in np.arange(1, 176,2).tolist()]` consists of the following parts:

1) **`np.arange(1, 176, 2)`**: [Numpy function `np.arange()`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) returns evenly spaced values within a given range. In our example, we specify that we need a list of numbers between 1 and 176 (in the first two parameters of the function). Then, in the third parameter, we specify that the differences between two neighboring numbers (also called steps or spacing values) should be 2. The function itself will return an [array](https://www.w3schools.com/python/python_arrays.asp) of numbers that looks like [1, 3, 5, ..., 173, 175].

2) **`str(x).zfill(3)`**: Note that the numbers we generated by using `np.arange()` are not exactly in the format we need. In Python, `1` and `'001'` are two completely different objects. `1` is a one-digit integer, while `'001'` is a three-digit string. The two zeros in `'001'` are called leading zeros. To convert `1` to `'001'`, we first need to use the `str()` function to convert `1` to string type and then use the `zfill()` function to add leading zeros. The number we specify within the `zfill()` function indicates how many digits the resulting number should have. You can check the [documentation of `zfill()`](https://www.w3schools.com/python/ref_string_zfill.asp) for more examples.

3) **List comprehension**: The code `str(x).zfill(3)` can only be applied to one number at a time. To convert all the numbers in array [1, 3, 5, ..., 173, 175], we need to apply a mini loop, which is called list comprehension. A list comprehension is a concise way to create lists. It consists of an expression followed by a `for` clause. In this case, the expression is str(x).zfill(3) and the for clause iterates over the values generated by np.arange(1, 176, 2). This expression is essentially creating a list of strings where each number generated by np.arange(1, 176, 2) is converted to a string and then padded with zeros to ensure it's at least 3 characters long.

In [ ]:
#Generate the list of Ohio county fips code
county_fips = [str(x).zfill(3) for x in np.arange(1, 176,2)]

#See the list
county_fips

Next, we need to loop through all 88 county FIPS codes to get each county's monthly unemployment rates. To initiate a loop, we use the code `for c in county_fips:`. This loop iterates through each element in `county_fips`, assigning each element to the variable `c` in each iteration. For example, in the first iteration, `c` is assigned the value `'001'`. The most important part inside of the loop is the `bls.get_series` function. Here, we change the series id part to `'LAUCN39' + c + '0000000003'`. This allows Python to replace the county FIPS code part with a new value during each iteration. You can check the [`for` loop documentation](https://www.w3schools.com/python/python_for_loops.asp) to practice more with simple `for` loop commands.

The rest of the code is to transform the DataFrame to a format that is eaiser for us to work with:

1) **`oh_cnty_ur_df = pd.DataFrame()`**: This line of code creates an empty DataFrame `oh_cnty_ur_df` for us to save the data we pull by using API.

2) **`.rename(columns = {})`**: As you have seen in the previous example, after we pull the data, the column name for the unemployment rate is the series id. We should rename the column so that it is easier for us to combine the data and to avoid confusion.

3) **`ur_temp_df['cnty_fips'] = c`**: Since the data is at county-month level, every time we pull a county's data, we should add an indicator for that county so that later we can tell which county a data point is from. 

4) **Append data with the `pd.concat()` function**: During each iteration, after we pull a county's data and save it in a temporary DataFrame `ur_temp_df`, we need to add it to the data we pulled from previous iterations, i.e., `oh_cnty_ur_df`. In this case, we use a data manipulation technique called **append data**. This technique allows us to add rows from one dataset to another dataset. The example below shows how the final data looks like if we append Adams County's and Allen County's data. To append datasets, we use the `pd.concat()` function. Inside of the function, we need to define a list of DataFrame names (`[oh_cnty_ur_df, ur_temp_df]`). You can check the [`pd.concat()` documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) for more information.

<h6><center>Adams County Data</center></h6>

| date | unemployment rate | cnty_fips |
| :-: | :-: | :-: |
| 2020-01 | 9.5 | 001 |
| 2020-02 | 8.4 | 001 |
| 2020-03 | 9.1 | 001 |

<h6><center>Allen County Data</center></h6>

| date | unemployment rate | cnty_fips |
| :-: | :-: | :-: |
| 2020-01 | 5.3 | 003 |
| 2020-02 | 4.8 | 003 |
| 2020-03 | 5.5 | 003 |

<h6><center>Append Adams County and Allen County Data</center></h6>

| date | unemployment rate | cnty_fips |
| :-: | :-: | :-: |
| 2020-01 | 9.5 | 001 |
| 2020-02 | 8.4 | 001 |
| 2020-03 | 9.1 | 001 |
| 2020-01 | 5.3 | 003 |
| 2020-02 | 4.8 | 003 |
| 2020-03 | 5.5 | 003 |

Now you have all 88 Ohio counties' monthly unemployment rates from 2020 to 2023. Think about how you can use similar codes to get additional years of data, data from other states, or a different series of data. 

><font color='red'> **Before you run the code below, please change YOUR KEY to your own BLS key.** </font>

In [ ]:
#Generate an empty DataFrame
oh_cnty_ur_df = pd.DataFrame()

#Get unemployment and labor force from BLS website
for c in county_fips:
    
    #Get Unemployment Rate
    ur_temp_df = pd.DataFrame(bls.get_series('LAUCN39' + c + '0000000003',
                                             2020,
                                             2023, 
                                             key = 'YOUR KEY')).reset_index()
    
    #Rename the series ID to "unemployment rate"
    ur_temp_df = ur_temp_df.rename(columns = {'LAUCN39' + c + '0000000003' : 'unemployment_rate'})
    
    #Add a new column to store county fips code
    ur_temp_df['cnty_fips'] = c
    
    #Append DataFrames
    oh_cnty_ur_df = pd.concat([oh_cnty_ur_df, ur_temp_df])

#See the first five rows of the DataFrame
oh_cnty_ur_df.head()

Similar to the previous example, we should save the data on our computers so that we do not need to pull the data over and over again. 

><font color='red'> **Before you run the code below, make sure that you have changed data_directory to your own directory in the previous section (the place where you want to put the data on your laptop).**

In [ ]:
#Save the data
oh_cnty_ur_df.to_csv(data_directory + 'oh_cnty_ur.csv', index = False)

## 3. Census API Example
<a id = "census_api"> </a>

[Go back to Table of Content](#toc)

In this example, we will pull state level foreign-born population and Ohio school-age population at school district level by using Census API. Similar to the BLS example, before we can use the API, we need to have two things ready:

1) **A Census API key**: [Click here to register the key](https://api.census.gov/data/key_signup.html)

2) **The column name of the data you want to pull**. The column name of the Census data is similar to the series id for the BLS data. It uniquely identifies a series of data. There are two ways you can find the column name:
- The first way is to search for the data on the [census data website](https://data.census.gov/). Search for the key words. For example, you can type "foreign born" in the search engine and wait for the options to show up (as shown in the picture below). Then choose "S0501: Selected Characteristics of the Native and Foreign-Born Populations". Then download the data as a ZIP file. In the ZIP file, find the "Column Metadata" csv file. Search "foreign born" in the Metadata file. Finally, you can find the "Estimate!!Foreign born!!Total population" and its column name **"S0501_C03_001E"**.


- The second way is to search the data you want on [Census API document](https://api.census.gov/data/2020/acs/acs5/subject/variables.html). This document is very long though and is specifically for the 2020 five-year estimates data. When you search for "foreign born" in this document, you will get thousands of results. It would be easier if you know the specific table you need, which is **"S0501"** in this example. You will find the same information as what you find in the metadata file.

No matter which method you use, you should thoroughly research the data you need, such as the topics, the geography level available, the years you need, and the types of survey data you need. For most of the analyses, you should use the five-year estimates, since it is relatively more accurate and contains more information at more granular geographic levels than the one-year estimate and three-year estiamtes. You can read the [five-year estimates documentation](https://www.census.gov/data/developers/data-sets/acs-5year.2022.html#list-tab-1806015614) to learn more.

<h6><center> Search for "Foreign born" on Census Data Website</center></h6>
<div>
<img src = "foreign_born_census.png" width="450"/>
</div>

<h6><center> Column MetaData File</center></h6>
<div>
<img src = "foreign_born_census_column.png" width="450"/>
</div>

<h6><center> Census API Document</center></h6>
<div>
<img src = "foreign_born_census_api_doc.png" width="600"/>
</div>

### 3.1. State Level Foreign-born Population Percentage Change Between 2010 and 2020
<a id = "fb_pop"> </a>

[Go back to Table of Content](#toc)

To use the census API functions, we need to save our Census API key into an oject first. In the first line of the code, `api_key = Census('YOUR CENSUS API KEY')`, you should put your census API key into the `Census()` function as a string and save it to the object `api_key`. This way, we can easily use the key without having a long list of strings in our code later.

Now, we can pull the census data by using `api_key.acs5st.state()`. This function has three components:
- **`api_key`**: This is the Census API key object we defined in the previous step. You should always include it at the beginning of each census API function.
- **`acs5st`**: This represents the survey we want to pull the data from, which is American Community Survey Five-year Estimates Subject Tables. If you want to pull data from other surveys, you can change this part accordingly. For example, `acs5dp` represents American Community Survey Five-year Estimates Data Profiles and `acs3` represents American Community Survey Three-year Estimates. You can check the Datasets section in the [census API library](https://pypi.org/project/census/) documentation for more information.
- **`state`**: This part represents the data's geography level. Here, we want to pull state level data. You can change this part to other geography levels you need, such as county level (`state_county`) or zip code level (`state_zipcode`). However, due to data limitations, not all geography levels are avaialble for all surveys. Also, even if a geography level is available, due to smaller sample sizes for more granular geography levels, data in some areas may be suppressed. For example, if you search for Ohio foreign-born population at county level, not all counties' data are available to you. Please check the Geographies section in the [census API library](https://pypi.org/project/census/) documentation for more information.

Inside of the `api_key.acs5st.state()` function, we need to define three parameters:
- **`('Name', 'S0501_C03_001E')`**: The column names of the data series you want to pull. It needs to be a string.
- **`Census.ALL`**: You need to specify which state's data you need in the second parameter. However, since we need all states' data in this case, we can just use `Census.ALL` to indicates that we need all states' data. If you only need Ohio data, you can replace `Census.ALL` with `states.OH.fips`.
- **`year = 2010`**: The last parameter indicates which years of data you need. We pull the 2010 and 2020 data separately and save them to two DataFrames


><font color='red'> **Before you run the code below, please change YOUR CENSUS API KEY to your own Census API key.** </font>

In [ ]:
#Census API key
api_key = Census('YOUR CENSUS API KEY')

#Pull 2010 data
fb_pop_2010_df = pd.DataFrame(api_key.acs5st.state('S0501_C03_001E', 
                                                   Census.ALL,
                                                   year = 2010))

#Rename column
fb_pop_2010_df = fb_pop_2010_df.rename(columns = {'S0501_C03_001E' : 'fb_2010'})

#See the data
print(fb_pop_2010_df.head()) #By using print(), we can see the results, even if the code is in the middle of a code chunk.

#Pull 2020 data
fb_pop_2020_df = pd.DataFrame(api_key.acs5st.state('S0501_C03_001E', 
                                                   Census.ALL,
                                                   year = 2020))

#Rename column
fb_pop_2020_df = fb_pop_2020_df.rename(columns = {'S0501_C03_001E' : 'fb_2020'})

#See the data
print(fb_pop_2020_df.head())

Now our data are in two DataFrames, `fb_pop_2010_df` and `fb_pop_2020_df`. It would be easier for us to calculate percentage change if the two columns `fb_2010` and `fb_2020` are in the same DataFrame. In this case, we need to use another data manipulation technique, **Merge data**. When we appending two DataFrames, we combine them *vertically*, i.e., data points that are in the same columns are combined vertically. When we merging two DataFrames, we combine them *horizontally*, i.e., rows that have the same values in one or more common columns are combined together. In our example, `state` is the common column that is in both DataFrames. Thus, we can use the `.merge()` fuction to merge `fb_pop_2010_df` with `fb_pop_2020_df`. Inside of the function, we need to specify the DataFrame that we want to merge `fb_pop_2010_df` with and the common column we want to merge the data on (`on = 'state'`).

<h6><center>2010 State Level Foreign-Born Population</center></h6>

| NAME | fb_2010 | state |
| :-: | :-: | :-: | 
| Alabama | 157935 | 01 | 
| Alaska | 49762 | 02 | 
| Arizona | 884625 | 04 |

<h6><center>2020 State Level Foreign-Born Population</center></h6>

| fb_2020 | state |
| :-: | :-: | 
| 168053 | 01 | 
| 57342 | 02 | 
| 947611 | 04 |

<h6><center>Merge 2010 and 2020 State Level Foreign-Born Population</center></h6>

| NAME | fb_2010 | state | fb_2020 |
| :-: | :-: | :-: | :-: | 
| Alabama | 157935 | 01 | 168053 |
| Alaska | 49762 | 02 | 57342 |
| Arizona | 884625 | 04 | 947611 |

In [ ]:
#Combine the two datasets
fb_pop_df = fb_pop_2010_df.merge(fb_pop_2020_df, on = 'state')

#Check the final data
fb_pop_df.head()

Finally, we can use a simple math equation to calculate percentage change in foreign-born population at state level. We just need to divide the change in foreign-born population between 2010 and 2020 (`fb_pop_df['fb_2020'] - fb_pop_df['fb_2010']`) by foreign-born population in 2010. The results are rounded to three digits after decimal.

In [ ]:
#Calculate percentage change
fb_pop_df['fb_pct_chg'] = round((fb_pop_df['fb_2020'] - fb_pop_df['fb_2010']) / fb_pop_df['fb_2020'], 3)

#Check the data
fb_pop_df.head()

### 3.2. Ohio School District Level Population Under 18 Years
<a id = "schl_dist_pop18"> </a>

[Go back to Table of Content](#toc)

In the previous examples, we used Python libraries that are already available on the internet. While these libraries provide convenient and easy-to-understand functions for us to use, they do have limitations. For example, the census library does not include all the geographic levels and the most recent years of data. In this case, we need to write our own code to pull data via API.

As we discussed in the class, using API to pull data is similar to ordering grocery delivery. To have groceries delivered to your door, you first need to make an order. Then the store staff packs the groceries and deliver them to your door. In the context of using API, you first need to create your order of data by writing a search query URL. This is what we do in the first line of code below. It may seem long and scary, but let's dissect it.

- **Dataset**: The part <font color='blue'>**api.census.gov/data/2022/acs/acs5/subject**</font> specifies that we need 2022 ACS five-year estimates subject tables.
- **Data Series/Column Names**: Then we need to specify the columns (or data series) we want by using <font color='blue'>**?get=NAME,S0101_C01_022E**</font>. Here, we ask for two columns: 1) school district names (`NAME`) and 2) population under 18 years (`S0101_C01_022E`).
- **Gegraphy Level**: The next part is to define the geography level we want, <font color='blue'>**&for=school%20district%20(unified):*&in=state:39**</font>. The `*` in the middle indicates we want to pull the data of all the school districts in Ohio. `state:39` is the FIPS code for Ohio.
- **Census API Key**: Whenever you use census API, you need to use your API key. Here, we define it in the last part of the search query URL <font color='blue'>**&key=YOUR_CENSUS_API_KEY**</font>

Next, you need to make your data order and ask for a data delivery. We can achieve these steps by using the code `requests.get(url)`. This code sends a request to census API and retrieves the response from the API. The response includes the data in [JSON format](https://en.wikipedia.org/wiki/JSON). We can extract the JSON data by using `response.json()`. Finally, we use `pd.DataFrame()` to convert the data to the DataFrame format in Python. The first row of the JSON data is column name. That's why we use `data[1:]` to specify that the actual data starts from row 1 and use `columns = data[0]` to define the column names.

**You don't need to write the whole search query URL yourself.** The Census website provides a list of URL examples for different geography levels. Here is the list of examples for [2022 ACS five-year estimates](https://api.census.gov/data/2022/acs/acs5/examples.html). You can find URL examples for other years and other surveys in [Census API documentation](https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html#list-tab-559651575)

><font color='red'> **Before you run the code below, please change YOUR_CENSUS_API_KEY to your own Census API key (Scroll all the way to the right).** </font>

In [ ]:
# Write the search query url for 2022 OH school district data
url = 'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,S0101_C01_022E&for=school%20district%20(unified):*&in=state:39&key=YOUR_CENSUS_API_KEY'
    
# Send a request to Census API and retrieve the response
response = requests.get(url)

#Get JSON data
data = response.json()
    
#Convert data to DataFrame
oh_schl_dist_pop18_df = pd.DataFrame(data[1:], columns = data[0])

#Check the data
oh_schl_dist_pop18_df

### Class Activity 2
Work in groups. Either use the column metadata file or use the [census documentation](https://api.census.gov/data/2020/acs/acs5/subject/variables.html). to find the column name for foreign-born population 25 years or over with bachelor's degree. Then use the API code (either the `census` library or the search query URL method) to pull 2020 data.

## 4. Visualizations
<a id = "viz"> </a>

[Go back to Table of Content](#toc)

In this section, I will present two examples to enhance your visualization skills in Python. In the first example, we will create a line chart using the Matplotlib library. I will demonstrate how to incorporate additional features into the line chart, such as adding labels for the x-axis and y-axis and adjust the styles of the visualization. In the second example, I will illustrate how to create an interactive map with the plotly library.

### 4.1. Ohio County Unemployment Rates Over Time
<a id = "oh_cnty_line"> </a>

[Go back to Table of Content](#toc)

In this example, we will use the data we pulled in section 2.2. If you have saved it on your computer, you should use `pd.read_csv()` to import the data first. In the code below, we include an additional parameter, `dtype = {'cnty_fips' : 'str'}`. `dtype` is the parameter that tells Python the type of a column. For example, since `cnty_fips` only contains numbers, when Python read the data from the csv file, the default type of `cnty_fips` is integer. However, in this example, we want `cnty_fips` to be three-digit strings. Thus, we can use `dtype` and [the dictionary](https://www.w3schools.com/python/python_dictionaries.asp), `{'cnty_fips' : 'str'}` (similar to what we used in the `rename` function), to define the type of `cnty_fips`.

> <font color='red'>**Before you run the code below, make sure that you have changed data_directory to your own directory in section 2.1. You should make sure that you have run the code in section 2.2 to pull the data and have saved the data in the correct folder.**

In [ ]:
#Import the data if you have already pulled it by using API.
oh_cnty_ur_df = pd.read_csv(data_directory + 'oh_cnty_ur.csv', dtype = {'cnty_fips' : 'str'})

#Check the data
oh_cnty_ur_df.head()

Before we create the visualization, we need to pre-process our data so that it is easier for us to make the line charts. First, we should check the types of columns in `oh_cnty_ur_df`. We can see that the type of the `date` column is object (or Period if you use the data pulled directly from API and skip the `pd.read_csv()` part). Similar to what we did in the "Basics in Python" notebook, we should transform it to the datetime format. Second, we need to divide `unemployment_rate` by 100 so that its range is between 0 and 1. Finally, there are 88 counties in Ohio. We probably don't want to (and should not) include 88 lines on one single graph. Depending on what your main research question is, you should select the key information to present on one graph. Here, assume we want to compare the three main urban counties in Ohio. We can look up the county FIPS codes for Cuyahoga, Franklin, and Hamilton counties, and then use the `isin()` function to save their data to three DataFrames.

In [ ]:
#Check column types
oh_cnty_ur_df.info()

> In the code `oh_cnty_ur_df['date'] = pd.to_datetime(oh_cnty_ur_df['date'].astype('str'), format = '%Y-%m')`, note that I add `astype('str')`. This is because `pd.to_datetime` can only transform string variables. If your data is pulled from API and if you did not read it again from csv file, the `date` column would be Period type. In this case, you need the `astype('str')` to transform it to string type first.

In [ ]:
#Convert the date column to datetime format. 
#The .astype('str') function is to convert the column type to string in case you skip the pd.read_csv() part
oh_cnty_ur_df['date'] = pd.to_datetime(oh_cnty_ur_df['date'].astype('str'), format = '%Y-%m')

#Transform the unemployment rate 
oh_cnty_ur_df['unemployment_rate'] = round(oh_cnty_ur_df['unemployment_rate']/100, 3)

#Check the data now
oh_cnty_ur_df.head()

> The code `oh_cnty_ur_df[oh_cnty_ur_df['cnty_fips'].isin(['035'])]` is equivalent to `oh_cnty_ur_df[oh_cnty_ur_df['cnty_fips'] == '035]`, which is the format we used in the "Basics in Python" notebook.

In [ ]:
#Save Cuyahoga County, Franklin County, and Hamilton County data to three DataFrames
cuyahoga_ur_df = oh_cnty_ur_df[oh_cnty_ur_df['cnty_fips'].isin(['035'])]
franklin_ur_df = oh_cnty_ur_df[oh_cnty_ur_df['cnty_fips'].isin(['049'])]
hamilton_ur_df = oh_cnty_ur_df[oh_cnty_ur_df['cnty_fips'].isin(['061'])]

#Check one county's data to ensure it's in the correct format
cuyahoga_ur_df.head()

Now, we can create a line chart to compare unemployment rates in Cuyahoga, Franklin, and Hamilton counties between 2020 and 2023. The code for creating the line chart is still `plt.plot()`, except that in this example, I add an additional parameter, `marker = 'o'`, so that all the data points on the lines are marked by dots/stars/triangles. The Matplotlib library offers a variety of [markers](https://matplotlib.org/stable/api/markers_api.html) for your to label data points. 

The rest of the visualization code may seem long, but you can reuse most of them when creating visualizations in Python.

- **`plt.rcParams['figure.dpi'] = 200`** and **`plt.rcParams['savefig.dpi'] = 200`**: You may find that the graphs in "Basics in Python" are not very clear. You can actually define the definition of a graph in Python. The larger the number, the clearer the graph.


- **`plt.style.use('seaborn')`**: This line of code is to change the style of your graph. You can check the [documentation](https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html) to see how different styles look like, or type 
`plt.style.available` in a code cell to see all the styles that are available.


- **`fig, ax = plt.subplots(figsize = (8, 5))`**: The `plt.subplots()` creates a figure and a set of subplots. The figure is saved in the object `fig` and the subplot object is `ax`. This way, we can easily change or add features to our plots. In this example, we only have one plot (or subplot). If you are curious about multiple subplots, you can check [this example](https://matplotlib.org/stable/gallery/lines_bars_and_markers/errorbar_subsample.html#sphx-glr-gallery-lines-bars-and-markers-errorbar-subsample-py). We also define the size of the plot by using `figsize = (8, 5)`.


- **`ax.set_ylim([0, 0.25])`**: Sometimes, you may need to adjust the range of your y-axis (or x-axis). In this case, you can use `set_ylim()` (or `set_xlim()`). `ax` referes to the subplot object we created earlier. You can check the [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html) here for more information and more examples. 


- **`ax.set_ylabel()`** and **`ax.set_xlabel()`**: This code is to add a label to explain what the values on your y-axis or x-axis mean. Here, we also use `fontsize = 12` to adjust the text size. You can check the [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylabel.html) here for more information and more examples. 


These are not all the features Matplotlib allows you to add. In the later notebooks, we will see additional code. Again, the visualization code in Python may seem to be long, but once you understand its structure, you can easily reuse and adjust it for other visualizations.

In [ ]:
#Improve the definition of the graph
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

#Set a style for the graph. You can use 'plt.style.available' to check what stypes are available
plt.style.use('seaborn')

#Save the figure and subplot to objects fig and ax. Also define the size of the plot.
fig, ax = plt.subplots(figsize = (8, 5))

#Plot the line chart.
#Here, we define x axis to be the reporting period and y axis to be unemployment rate
plt.plot(cuyahoga_ur_df['date'], cuyahoga_ur_df['unemployment_rate'], label = 'Cuyahoga', marker = 'v')
plt.plot(franklin_ur_df['date'], franklin_ur_df['unemployment_rate'], label = 'Franklin', marker = '*')
plt.plot(hamilton_ur_df['date'], hamilton_ur_df['unemployment_rate'], label = 'Hamilton', marker = 'o')

#Set the range of values on y axis
ax.set_ylim([0, 0.25])

#Add labels to x-axis and y-axis
ax.set_ylabel('Unemployment Rate', fontsize = 12)
ax.set_xlabel('Month' , fontsize = 12)

#Show legend
plt.legend()

#Show the plot in the notebook
plt.show()

### Class Activity 3
Work in groups. Create a line chart for three different counties of your choice. Discuss with each other what additional features you can include in a line chart.

### 4.2.* Percentage Change in Foreign-Born Population
<a id = "fb_map"> </a>

[Go back to Table of Content](#toc)

In this example, we will use the data we pulled in section 3.1, `fb_pop_df`. Ideally, you should save your data to a csv file after you pull it by using API. However, since we only have two years of state level data in this case, we will skip this step here. First, let's check if our data has all the elements we need in order to create a map. Note that the only indicator for state is the column `state`, which is the two-digit state FIPS code. Depending on the software/library you use to create the map, state FIPS code may or may not be sufficient. We will use the Plotly library `px.choropleth()` function to create the map in this example. The function only recognizes state abbreviations, such as "OH".

The easiest way to convert the two-digit state FIPS codes to state abbreviations is to merge our data with a **crosswalk**, such as the **"us_state_fips.csv"** file. The crosswalk has three columns: 1) state names, `stname`; 2) state FIPS codes, `state`; and 3) state abbreviations, `stusps`. As long as your data has one of these columns, you can use the crosswalk to convert it to the other two formats. In our example, we just need to merge `fb_pop_df` with the crosswalk `fips_xwalk_df` on the common column `state`. Then our new DataFrame, `fb_pop_st_df`, will contain state abbreviations.

> <font color='red'>**Before you run the code below, make sure that you have changed data_directory to your own directory in section 2.1. You should make sure that you have run the code in section 3.1 to pull the data and have saved the data in the correct folder.**

In [ ]:
#Check the data we pulled in section 3.1
fb_pop_df.head()

In [ ]:
#Read in the crosswalk file
fips_xwalk_df = pd.read_csv('us_state_fips.csv', dtype = {'state' : 'string'})

#Merge the crosswalk file with `fb_pop_df`
fb_pop_st_df = fb_pop_df.merge(fips_xwalk_df, on = 'state')

#Check the new DataFrame
fb_pop_st_df.head()

Now, we can use the `px.choropleth()` function to create a choropleth map. Similar to the previous example, the code seems to be long, but you just need to understand its structure. 

- **`fb_pop_st_df`**: The first parameter is the DataFrame that contains the data you use to create the map.


- **`locations = 'stusps'`**: The second parameter tells Python which column of the DataFrame contains the state abbreviation.


- **`locationmode = 'USA-states`**: The `locationmode` parameter tells Python which set of locations it should use to match the values in `locations`. In this package, `locationmode` only has three values, `ISO-3`, `USA-states`, or `country names`.


- **`scope = 'usa'`**: This parameter defines the scope of the map. It can also be `world`, `europe`, `asia`, `africa`, `north america`, or `south america`.


- **`color = 'fb_pct_chg'`**: The color parameter defines the column based on which the color should change. For example, darker colors could represent states that have higher percentage changes in foreign-born population.


- **`labels = {'fb_pct_chg' : '%Change'}`**: The default setting of the visualization is to use column names as the legend/axis titles. However, the column names are usually not reader-friendly. In this case, we can use `labels = {}` to replace column names with more formal descriptions.


- **`color_continuous_scale = 'Viridis_r'`**: You can select the color palette for your map (or any visualizations). Check the documentation here for [Plotly color palette](https://plotly.com/python/builtin-colorscales/) and the documentation here for [Matplotlib color palette](https://matplotlib.org/stable/users/explain/colors/colormaps.html).

In [ ]:
#Create the interactive map
fig = px.choropleth(fb_pop_st_df, #Define the Dataframe we will use
                    locations = 'stusps', #The column that contains state abbreviations
                    locationmode = 'USA-states', #The set of locations that used to match the values in `locations`.
                                                 #It can only be ‘ISO-3’, ‘USA-states’, or ‘country names’ 
                    scope = 'usa', #The scope of the map
                    color = 'fb_pct_chg', #The name of the column that contains the values for assigning colors
                    labels = {'fb_pct_chg' : '%Change'}, #Use more reader-friendly descriptions as the legend title
                    color_continuous_scale = 'Viridis_r') #Assign color palette

#Show the graph
fig.show()